<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# SVM
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, average_precision_score, classification_report

Here's the plan. 

To train the model, we're going to combine a BUNCH of data. Data from Meander && Spiral from Old Hand PD and New Hand PD. Just to see how it goes. Then we'll try more detailed extraction.

Also, try only on new with age?

Run simple SVM based on output of neural network model and age and some other stats :)

### Extracting from Old Hand PD

In [3]:
spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Spiral_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [4]:
spiral_df.shape

(368, 10)

In [5]:
spiral_df

,Label,RMS,Max_dist,Min_dist,SD,MRT,Max_ET,Min_ET,SD_ET,HT_ET_Diff
Image,,,,,,,,,,
1,1,3521.258301,6247.052734,30801.99219,0.014133,26.785328,176.600113,0.002130,1781.795898,0.250000
2,1,4098.876465,6032.535156,34369.70313,0.022838,26.529615,168.352737,0.084960,1443.217529,0.273585
3,1,3854.601807,6453.114746,34709.44531,0.000251,23.670755,180.898300,0.009303,1621.750000,0.256329
4,1,4069.221924,6844.231445,32181.26367,0.000168,23.456329,179.116043,0.021419,1454.390137,0.249221
5,1,4104.271973,6949.925293,36444.95313,0.004731,22.488258,188.256210,0.000000,1553.536499,0.214511
...,...,...,...,...,...,...,...,...,...,...
364,2,5593.215820,7997.703613,36371.22656,0.000190,26.964298,179.071930,0.059952,1885.214478,0.198653
365,2,7986.296387,7497.133789,29909.39063,2.613612,22.278151,203.759109,0.033372,1667.547974,0.241379
366,2,4652.542969,7275.474121,39475.41406,0.240567,21.622019,198.719940,0.000000,1469.676147,0.300000


In [6]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [7]:
spiral_df.dtypes

Label           int64
RMS           float64
Max_dist      float64
Min_dist      float64
SD            float64
MRT           float64
Max_ET        float64
Min_ET        float64
SD_ET         float64
HT_ET_Diff    float64
dtype: object

In [8]:
# extracting necessary columns from spiral df
X_spiral = spiral_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_spiral = spiral_df['Label']

y_spiral = pd.DataFrame(y_spiral)

In [9]:
meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/Meander_HandPD.txt', delimiter=' ', index_col=0, names=['Image', 'Label', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [10]:
meander_df.shape

(368, 10)

In [11]:
spiral_df['Label'].value_counts()

2    296
1     72
Name: Label, dtype: int64

In [12]:
# choosing all patients from meander_df
X_meander = meander_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]

y_meander = meander_df['Label']
y_meander = pd.DataFrame(y_meander)

### Extracting from New HandPD

In [13]:
new_spiral_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewSpiral.csv', skiprows = 1, index_col = 0, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']) # skiprows to remove first row of old columns names

In [14]:
new_spiral_df.columns

Index(['Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS',
       'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET',
       'HT_ET_Diff'],
      dtype='object')

In [15]:
# extracting necessary columns from spiral df
X_new_spiral = new_spiral_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_new_spiral = new_spiral_df['Label']

y_new_spiral = pd.DataFrame(y_new_spiral)

In [16]:
X_test = np.apply_along_axis(pd.to_numeric, 1, X_new_spiral)

In [17]:
new_meander_df = pd.read_csv('/content/drive/My Drive/Data/HandPD-Replication/NewMeander.csv', index_col = 0, skiprows =1, names=['ID_EXAM', 'Image', 'ID_PATIENT', 'Label', 'GENDER', 'HANDEDNESS', 'AGE', 'RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff'])

In [18]:
# extracting necessary columns from meander df
X_new_meander = new_meander_df[['RMS', 'Max_dist', 'Min_dist', 'SD', 'MRT', 'Max_ET', 'Min_ET', 'SD_ET', 'HT_ET_Diff']]
y_new_meander = new_meander_df['Label']

y_new_meander = pd.DataFrame(y_new_meander)

##**Preprocessing**

###**Normalization**

fi' = (fi - avg)/std

In [19]:
# normalization with the paper's method (formula above)
def feature_normalization(df):
  avg_dev = df.mad(axis = 0)
  std_dev = df.std(axis = 0)

  df = df.sub(avg_dev)
  df = df.divide(std_dev)

  return df

In [20]:
# going to normalize together
X = pd.concat([X_spiral, X_new_spiral,X_meander, X_new_meander])
y = pd.concat([y_spiral, y_new_spiral, y_meander, y_new_meander])

In [21]:
X.shape

(1264, 9)

In [23]:
y.value_counts()

Label
2        840
1        424
dtype: int64

In [24]:
# normalizing X
X = feature_normalization(X)

In [25]:
X.shape

(1264, 9)

###Train-Test-Split

In [26]:
# Train-Test Split, 80% train, 20% test, stratification across PD diagnosis (label)
y_label = y.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.2, stratify = y_label)

In [27]:
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)

In [28]:
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

###**Quick Check**

In [29]:
y_train.value_counts()

Label
2        672
1        339
dtype: int64

In [30]:
y_test.value_counts() # relatively same 1:2 ratio -- 29:59 is almost equal as 85:168

Label
2        168
1         85
dtype: int64

##**SVM Implementation**

###**Training**

In [31]:
clf = SVC(kernel = 'rbf', probability = True, class_weight = 'balanced')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

To note, it may be useful to look at the weights for this model (see which features are most valuable), but these are only relevant for a linear kernel. RBF kernel does not have relevant/interpretable weights. And of course, linear kernel is not very helpful and good in predicting (51% accuracy). The features are not linearly separable.

In [32]:
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)

In [33]:
y_pred = pd.Series(y_pred)

##**Results**

In [34]:
clf.score(X_test, y_test)

0.7747035573122529

In [35]:
y_test.dtypes

Label    int64
dtype: object

In [36]:
target_names = ['Control', 'PD']
results = classification_report(y_test, y_pred, target_names = target_names, output_dict=True)
results = pd.DataFrame(results).transpose()
conf_mat = confusion_matrix(y_test, y_pred)

In [37]:
results

,precision,recall,f1-score,support
Control,0.759259,0.482353,0.589928,85.000000
PD,0.778894,0.922619,0.844687,168.000000
accuracy,0.774704,0.774704,0.774704,0.774704
macro avg,0.769077,0.702486,0.717307,253.000000
weighted avg,0.772298,0.774704,0.759096,253.000000


In [38]:
conf_mat

array([[ 41,  44],
       [ 13, 155]])

In [ ]:
TN, FP, FN, TP = conf_mat.ravel()

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)

# Specificity or true negative rate
TNR = TN/(TN+FP) 

# Precision or positive predictive value
PPV = TP/(TP+FP)

# Negative predictive value
NPV = TN/(TN+FN)

# Fall out or false positive rate
FPR = FP/(FP+TN)

# False negative rate
FNR = FN/(TP+FN)

# False discovery rate
FDR = FP/(TP+FP)

print("TP: ", TP)
print("TN: ", TN)
print("FP: ", FP)
print("FN: ", FN)

print("Sensitivity: ", TPR)
print("Specificity: ", TNR)
print("NPV: ", NPV)
print("PPV: ", PPV)

TP:  59
TN:  17
FP:  12
FN:  0
Sensitivity:  1.0
Specificity:  0.5862068965517241
NPV:  1.0
PPV:  0.8309859154929577


Can we somehow get the weights for the model to learn which features are considered most useful?